In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

#from IPython import get_ipython
#get_ipython().magic('reset -sf')

import torch
torch.cuda.is_available()


import tensorflow as tf
import autograd.numpy as np
import matplotlib.pyplot as plt
from sys import stdout
from autograd import grad
import time 


if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

devicecpu=torch.device("cpu")

#plt.close('all')

In [ ]:
#return result of an input after propagation in space F_2

def xtopsi2_torch(x):
    if random2 == "true":
        p=x.shape[1]  #train input dimension --> column vector with 100 rows
        np.random.seed(11)
        bias2=torch.from_numpy(0.1*np.random.randn(D2))  #b_2^2 with distribution 0.1 * N(0,1) 
        W2=torch.from_numpy(1.5*np.divide(np.random.randn(p, D2),np.sqrt(p)))   
        #W_2^2 with corresponding distribution
        e=torch.matmul(x,W2)+bias2
    else: 
        e=x
        
    if activ == "relu":
        e=torch.clamp(e, min=0)  #activation function ReLU a(.) = max(0,.)
    if activ == "sigmoid":
        e =  1/(1 + torch.exp(-e))
    if activ == "tanh":
        e = torch.tanh(e)
    
    if random2 == "false":
        e = torch.cat([e,torch.ones((x.shape[0],1))],dim=1)
    #e=np.cos(e)
    return torch.transpose(e, 0, 1)


In [ ]:
def getalpha2_torch(vx_train,y_train):   
    psi2_train=xtopsi2_torch(vx_train)  
    A=torch.matmul(psi2_train,torch.transpose(psi2_train,0,1)) + len(y_train)*gamma2*torch.eye(D2)
    b=torch.matmul(psi2_train,y_train)
    #alpha2=torch.linalg.solve(A,b)
    alpha2=torch.linalg.solve(A,b)
    loss2=(1/len(y_train))*torch.sum((torch.matmul(torch.transpose(psi2_train,0,1),alpha2) - y_train)**2)
    loss2=loss2+gamma2*torch.sum(alpha2**2)
    return (loss2,alpha2)

In [ ]:
#obtain the fitted values 
def getYpred(x,alpha2):
    psi2=xtopsi2_torch(x)
    pred=torch.matmul(torch.transpose(psi2,0,1), alpha2)
    return pred

In [ ]:
def xtopsi1_torch(x):
    if random1 == "true":
        np.random.seed(100)
        p=x.shape[1]  #train input dimension --> column vector with 100 rows
        bias1=torch.from_numpy(0.1*np.random.randn(D1))   #b_1 with distribution 0.1 * N(0,1) 
        W1=torch.from_numpy(1.5*np.divide(np.random.randn(p, D1),np.sqrt(p)))  #W_1 with corresponding distribution 
        e=torch.matmul(x,W1)+bias1
    else: 
        e=x
        
    if activ == "relu":
        e=torch.clamp(e, min=0)  #activation function ReLU a(.) = max(0,.)
    if activ == "sigmoid":
        e =  1./(1 + torch.exp(-e))
    if activ == "tanh":
        e = torch.tanh(e)
    
    if random1 == "false":
        e = torch.cat([e,torch.ones((x.shape[0],1))],dim=1)
    #e=np.cos(e)
    return torch.transpose(e, 0, 1)

In [ ]:
def phi_torch(x,alpha1):
    for i in range(L):
        xtopsi1_transp=torch.transpose(xtopsi1_torch(x),0,1)
        x=x+torch.matmul(xtopsi1_transp,alpha1[:,:,i])  #h is removed 
    return x

In [ ]:
from torch.autograd import Variable

def loss_torch(alpha1):
    loss1=0.5*L*torch.sum(alpha1*alpha1)
    phivx_train=phi_torch(vx_train,alpha1)
#    phivx_train=vx_train
    (loss2,alpha2)=getalpha2_torch(phivx_train,y_train)
    return nu*loss1+loss2



In [ ]:
def Error_plot(init_train_loss,final_train_loss,init_test_loss,final_test_loss,inloss,finloss,dataset):
    #perc_list = np.arange(10,90,10)
    #N_list = np.array([50,100,200,400,800,1600])
    #N_list = np.arange(10,110,10)
    Ntrain=5000
    N_list=[np.int(rate*Ntrain) for rate in np.arange(0.1,1.1,0.1)]
    fig = plt.figure(figsize=(16,4)) 
    c=1

    plt.subplot(1, 3, c)
    plt.title('Training Error')
    plt.plot(N_list,init_train_loss,label='Ridge regression')
    plt.plot(N_list,final_train_loss, label='Mechanical regression')
    #plt.plot(perc_list,ridge_loss_train,label='ridge')
    plt.xlabel(r'$N_{train}$')
    plt.ylabel('Error')
    #plt.legend()
    c+=1

    plt.subplot(1, 3, c)
    plt.title('Testing Error')
    plt.plot(N_list,init_test_loss,label='Ridge regression')
    plt.plot(N_list,final_test_loss,label='Mechanical regression')
    #plt.plot(perc_list,ridge_loss_test,label='ridge')
    plt.xlabel(r'$N_{train}$')
    plt.ylabel('Error')
    #plt.legend()
   # plt.yscale('log')
    c+=1

    plt.subplot(1, 3, c)
    plt.title(r"$L_2$ regularized loss")
    plt.plot(N_list,inloss,label='Ridge regression')
    plt.plot(N_list,finloss,label='Mechanical regression')
    plt.xlabel(r'$N_{train}$')
    plt.legend()
    #plt.ylabel('MSE')
    

    #plt.savefig('Error_{}_{}_Adam.pdf'.format(dataset,activ))
    plt.show()

In [ ]:
def optimize_alpha1(alpha1_init,nsteps=1000,lr=0.1):
    alpha1 = Variable(alpha1_init ,requires_grad=True)
    opt = torch.optim.Adam([alpha1],lr=lr)

    loss_history = []

    for i in range(nsteps):
        opt.zero_grad()
        stdout.write("\r[%s]" % (i+1)) 
        loss = loss_torch(alpha1) # Calculate loss function
        loss_history.append(loss.detach().numpy()) # Do some recordings for plots
        
        loss.backward()
        opt.step()
        
       # if i % 50 == 0:
       #     print(f'iter {i+1}: loss = {loss}')
    
    #print(f'final loss: loss = {loss}')

    print("Min loss after training alpha : {}".format(np.min(loss_history)))

    plt.figure()    
    plt.plot(loss_history)                   #plot the loss as a function of iteration i
    plt.title('N= {}, lr= {}'.format(len(vx_train),lr))
    plt.show()
    
    return alpha1 , loss_history

        #stdout.write("\r[%s]" % loss) 



In [ ]:
# define prediction for classification

def gettesterror_classif(pred,y):
    true_labels = np.argmax(y,axis=1)
    pred_labels=torch.argmax(pred,axis=1)
    correct=(pred_labels==true_labels)
    error=1-np.sum(correct.numpy())/len(y)
    return error



In [ ]:
def getYpred(x,alpha2):
    psi2=xtopsi2_torch(x)
    pred=torch.matmul(torch.transpose(psi2,0,1), alpha2)
    return pred

def get_error(x,y,alpha2):
    pred=getYpred(x,alpha2)
    true_labels=np.argmax(y,axis=1)
    pred_labels=torch.argmax(pred,axis=1)
    correct=(pred_labels==true_labels)
    error=1-np.sum(correct.numpy())/len(y)
    return error

## MNIST

In [ ]:
def Loaddata(rate):
    loadmnist=False
    Ntrain=np.int(rate*5000)
    Ntest=10000
    if loadmnist:
        mnist = tf.keras.datasets.mnist
        class_names = ['0', '1', '2', '3', '4',
                   '5', '6', '7', '8', '9']
    else:
        mnist = tf.keras.datasets.fashion_mnist
        class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train=x_train.astype('float64')
    x_test=x_test.astype('float64')
    x_train=x_train[0:Ntrain,:,:]
    y_train=y_train[0:Ntrain]
    x_test=x_test[0:Ntest,:,:]
    y_test=y_test[0:Ntest]
    Y_train=np.zeros((Ntrain,10))
    Y_test=np.zeros((Ntest,10))
    for i in range(10):
        Y_train[y_train[0:Ntrain]==i,i]=1
        Y_test[y_test[0:Ntest]==i,i]=1
    x_train, x_test = x_train / 255.0, x_test / 255.0
    normalizedata=False
    if normalizedata:
        a=np.sqrt(np.sum(x_train*x_train,(1,2)))
        a=a[:,None,None]+np.zeros((Ntrain,28,28))
        x_train = np.divide(x_train , a)
        a=np.sqrt(np.sum(x_test*x_test,(1,2)))
        a=a[:,None,None]+np.zeros((Ntest,28,28))
        x_test = np.divide(x_test , a)
        a=0 
    show=False
    if show == True:
        def showimages(x_train,y_train):
            plt.figure(figsize=(10,10))
            for i in range(25):
                plt.subplot(5,5,i+1)
                plt.xticks([])
                plt.yticks([])
                plt.grid(False)
                plt.imshow(x_train[i], cmap=plt.cm.binary)
                plt.xlabel(class_names[y_train[i]])
            plt.show()
        showimages(x_train,y_train)
    d2=28*28
    vx_train=np.reshape(x_train,(Ntrain,d2))
    vx_test=np.reshape(x_test,(Ntest,d2))
    return (vx_train,Y_train,vx_test,Y_test)



In [ ]:
loadmnist=False

def get_sets(rate):
    (vx_train,y_train,vx_test,y_test)=Loaddata(rate)

    vx_train=torch.tensor(vx_train.astype('float64'))
    vx_test=torch.tensor(vx_test.astype('float64'))
    y_train=torch.tensor(y_train.astype('float64'))
    y_test=torch.tensor(y_test.astype('float64'))

    return (vx_train, y_train), (vx_test, y_test)


### 1) MNIST 

In [ ]:
#%who

In [ ]:
#%store -r alpha1_mnist1L_tanh
#%store -r loss_mnist1L_relu_NEW 
#%store -r train_mnist1L_relu_NEW 
#%store -r test_mnist1L_relu_NEW 

In [ ]:
gamma2=0.01
nu=0.01


p = 784 # vx_train.shape[1]
D1 = p + 1
D2 = 784

Layers=[1,2,3]


random1="false"
random2="true"


# activ = relu

activ="relu"

alpha1min_list_fash_relu = []
loss_list_fash_relu = []
train_list_fash_relu = []
test_list_fash_relu = []



for l in Layers:
    
    L=l
    alpha1min_list = []
    loss_list = []
    train_loss = []
    test_loss = []
    opt_time = []


    for rate in np.arange(0.1,1.1,0.1):

        start = time.perf_counter()

        # get the data
        loadmnist=False 
        (vx_train, y_train), (vx_test, y_test) = get_sets(rate)

        Ntrain=vx_train.shape[0]
        Ntest=vx_test.shape[0]

        # optimize over alpha
        lr=0.001
        alpha1min, loss_history = optimize_alpha1(torch.zeros((D1,p,L),dtype=torch.float64),nsteps=200,lr=lr)
        lmin = loss_torch(alpha1min).detach().numpy()

        # append the loss / alpha
        loss_in=loss_torch(torch.zeros((D1,p,L),dtype=torch.float64)).numpy()
        loss_list.append([loss_in,lmin])
        alpha1min_list.append(alpha1min.detach().numpy())


        # append the train/test error
        _,alpha2min=getalpha2_torch(phi_torch(vx_train,alpha1min),y_train)
        _,alpha2_init=getalpha2_torch(vx_train,y_train)

        inierror_train=get_error(vx_train,y_train,alpha2_init)
        finerror_train=get_error(phi_torch(vx_train,alpha1min),y_train,alpha2min)

        inierror_test=get_error(vx_test,y_test,alpha2_init)
        finerror_test=get_error(phi_torch(vx_test,alpha1min),y_test,alpha2min)

        print('initial {} error= {}'.format("train",np.round(inierror_train,3))) 
        print('final {} error= {}'.format("train",np.round(finerror_train,3)))

        print('initial {} error= {}'.format("test",np.round(inierror_test,3)))
        print('final {} error= {}'.format("test",np.round(finerror_test,3)))

        # append the train/test error
        #inierror_train,finerror_train,inierror_test,finerror_test = get_error(vx_train,y_train,vx_test,y_test)

        train_loss.append(np.array([inierror_train,finerror_train]))
        test_loss.append((np.array([inierror_test,finerror_test])))

        end = time.perf_counter()
        print(f"Optimization time: {end-start}")


    alpha1min_list_fash_relu.append(alpha1min_list)
    loss_list_fash_relu.append(loss_list)
    train_list_fash_relu.append(train_loss)
    test_list_fash_relu.append(test_loss)
    

    
# activ = tanh  

activ="tanh"

alpha1min_list_fash_tanh = []
loss_list_fash_tanh = []
train_list_fash_tanh = []
test_list_fash_tanh = []



for l in Layers:
    
    L=l
    alpha1min_list = []
    loss_list = []
    train_loss = []
    test_loss = []
    opt_time = []


    for rate in np.arange(0.1,1.1,0.1):

        start = time.perf_counter()

        # get the data
        loadmnist=False 
        (vx_train, y_train), (vx_test, y_test) = get_sets(rate)

        Ntrain=vx_train.shape[0]
        Ntest=vx_test.shape[0]

        # optimize over alpha
        lr=0.001
        alpha1min, loss_history = optimize_alpha1(torch.zeros((D1,p,L),dtype=torch.float64),nsteps=200,lr=lr)
        lmin = loss_torch(alpha1min).detach().numpy()

        # append the loss / alpha
        loss_in=loss_torch(torch.zeros((D1,p,L),dtype=torch.float64)).numpy()
        loss_list.append([loss_in,lmin])
        alpha1min_list.append(alpha1min.detach().numpy())


        # append the train/test error
        _,alpha2min=getalpha2_torch(phi_torch(vx_train,alpha1min),y_train)
        _,alpha2_init=getalpha2_torch(vx_train,y_train)

        inierror_train=get_error(vx_train,y_train,alpha2_init)
        finerror_train=get_error(phi_torch(vx_train,alpha1min),y_train,alpha2min)

        inierror_test=get_error(vx_test,y_test,alpha2_init)
        finerror_test=get_error(phi_torch(vx_test,alpha1min),y_test,alpha2min)

        print('initial {} error= {}'.format("train",np.round(inierror_train,3))) 
        print('final {} error= {}'.format("train",np.round(finerror_train,3)))

        print('initial {} error= {}'.format("test",np.round(inierror_test,3)))
        print('final {} error= {}'.format("test",np.round(finerror_test,3)))

        # append the train/test error
        #inierror_train,finerror_train,inierror_test,finerror_test = get_error(vx_train,y_train,vx_test,y_test)

        train_loss.append(np.array([inierror_train,finerror_train]))
        test_loss.append((np.array([inierror_test,finerror_test])))

        end = time.perf_counter()
        print(f"Optimization time: {end-start}")


    alpha1min_list_fash_tanh.append(alpha1min_list)
    loss_list_fash_tanh.append(loss_list)
    train_list_fash_tanh.append(train_loss)
    test_list_fash_tanh.append(test_loss)
    
    
    

    
# activ = sigmoid  

activ="sigmoid"

alpha1min_list_fash_sigm = []
loss_list_fash_sigm = []
train_list_fash_sigm = []
test_list_fash_sigm = []


    
for l in Layers:
    
    L=l
    alpha1min_list = []
    loss_list = []
    train_loss = []
    test_loss = []
    opt_time = []


    for rate in np.arange(0.1,1.1,0.1):

        start = time.perf_counter()

        # get the data
        loadmnist=False 
        (vx_train, y_train), (vx_test, y_test) = get_sets(rate)

        Ntrain=vx_train.shape[0]
        Ntest=vx_test.shape[0]

        # optimize over alpha
        lr=0.001
        alpha1min, loss_history = optimize_alpha1(torch.zeros((D1,p,L),dtype=torch.float64),nsteps=200,lr=lr)
        lmin = loss_torch(alpha1min).detach().numpy()

        # append the loss / alpha
        loss_in=loss_torch(torch.zeros((D1,p,L),dtype=torch.float64)).numpy()
        loss_list.append([loss_in,lmin])
        alpha1min_list.append(alpha1min.detach().numpy())


        # append the train/test error
        _,alpha2min=getalpha2_torch(phi_torch(vx_train,alpha1min),y_train)
        _,alpha2_init=getalpha2_torch(vx_train,y_train)

        inierror_train=get_error(vx_train,y_train,alpha2_init)
        finerror_train=get_error(phi_torch(vx_train,alpha1min),y_train,alpha2min)

        inierror_test=get_error(vx_test,y_test,alpha2_init)
        finerror_test=get_error(phi_torch(vx_test,alpha1min),y_test,alpha2min)

        print('initial {} error= {}'.format("train",np.round(inierror_train,3))) 
        print('final {} error= {}'.format("train",np.round(finerror_train,3)))

        print('initial {} error= {}'.format("test",np.round(inierror_test,3)))
        print('final {} error= {}'.format("test",np.round(finerror_test,3)))

        # append the train/test error
        #inierror_train,finerror_train,inierror_test,finerror_test = get_error(vx_train,y_train,vx_test,y_test)

        train_loss.append(np.array([inierror_train,finerror_train]))
        test_loss.append((np.array([inierror_test,finerror_test])))

        end = time.perf_counter()
        print(f"Optimization time: {end-start}")


    alpha1min_list_fash_sigm.append(alpha1min_list)
    loss_list_fash_sigm.append(loss_list)
    train_list_fash_sigm.append(train_loss)
    test_list_fash_sigm.append(test_loss)






In [ ]:
#%store -r alpha1min_mnist1L_sigm_Norm

In [ ]:
def plot_error_NEW(loss_list,train_list,test_list):
    #N_list = np.array([50,100,200,400,800,1600])
    Layers=np.array([1,2,3])

    N_list=[np.int(rate*5000) for rate in np.arange(0.1,1.1,0.1)]
    fig = plt.figure(figsize=(15,4)) 
    c=1
    plt.rc('axes', prop_cycle=(cycler('color', ['tab:blue','tab:orange', 'tab:green', 'tab:red', 'tab:purple'])))
    
    plt.subplot(1, 3, c)
    plt.title('Training Error')
    plt.plot(N_list,np.array(train_list[0])[:,0],label='Ridge')
    for train,L in zip(train_list,Layers):
        plt.plot(N_list,np.array(train)[:,1],label=r'$L = {}$'.format(L))
    plt.xlabel(r'$N$')
    #plt.yscale('simlog')
    c+=1

    plt.subplot(1, 3, c)
    plt.title('Testing Error') 
    plt.plot(N_list,np.array(test_list[0])[:,0],label='Ridge')
    for test,L in zip(test_list,Layers):
        plt.plot(N_list,np.array(test)[:,1],label=r'$L = {}$'.format(L))
    plt.xlabel(r'$N$')
    #plt.yscale('log')
    c+=1

    plt.subplot(1, 3, c)
    plt.title(r"$L_2$ regularized loss")
    plt.plot(N_list,np.array(loss_list[0])[:,0],label='Ridge')
    for l,L in zip(loss_list,Layers):
        plt.plot(N_list,np.array(l)[:,1], label=r'$L = {}$'.format(L))
    plt.xlabel(r'$N$')
    #plt.yscale('log')

    plt.legend() #loc=(1.04,0.55)
    #plt.tight_layout()

    #activ="sigmoid"
    #dataset="MNIST_fashion"
    #plt.savefig('Error_{}_{}_NEW.pdf'.format(dataset,activ))
    
    



In [ ]:
plot_error_NEW(loss_list_fash_sigm,train_list_fash_sigm,test_list_fash_sigm)

In [ ]:
# Old version of error plots 


#activ="sigmoid"

#Error_plot(np.array(train_mnist1L_relu_small)[:,0],np.array(train_mnist1L_relu_small)[:,1],
#         np.array(test_mnist1L_relu_small)[:,0],np.array(test_mnist1L_relu_small)[:,1],
#         np.array(loss_mnist1L_relu_small)[:,0],np.array(loss_mnist1L_relu_small)[:,1],
#         "MNIST1L_lr0.001")

#perc_list = np.arange(10,90,10)
    #N_list = np.array([50,100,200,400,800,1600])
    #N_list = np.arange(10,110,10)
N_list=[np.int(rate*5000) for rate in np.arange(0.1,1.1,0.1)]
fig = plt.figure(figsize=(15,4)) 
c=1

plt.subplot(1, 3, c)
plt.title('Training Error')
plt.plot(N_list,np.array(train_mnist1L_sigm_Norm)[:,0],label='Ridge')
plt.plot(N_list,np.array(train_mnist1L_sigm_Norm)[:,1], label=r'$L = 1$')
plt.plot(N_list,np.array(train_mnist2L_sigm_Norm)[:,1], label=r'$L = 2$')
plt.plot(N_list,np.array(train_mnist3L_sigm_Norm)[:,1], label=r'$L = 3$')
plt.xlabel(r'$N$')
#plt.yscale('simlog')
c+=1

plt.subplot(1, 3, c)
plt.title('Testing Error') 
plt.plot(N_list,np.array(test_mnist1L_sigm_Norm)[:,0],label='Ridge')
plt.plot(N_list,np.array(test_mnist1L_sigm_Norm)[:,1], label=r'$L = 1$')
plt.plot(N_list,np.array(test_mnist2L_sigm_Norm)[:,1], label=r'$L = 2$')
plt.plot(N_list,np.array(test_mnist3L_sigm_Norm)[:,1], label=r'$L = 3$')
plt.xlabel(r'$N$')
#plt.ylabel("Error")
#plt.yscale('log')
c+=1

plt.subplot(1, 3, c)
plt.title(r"$L_2$ regularized loss")
plt.plot(N_list,np.array(loss_mnist1L_sigm_Norm)[:,0],label='Ridge')
plt.plot(N_list,np.array(loss_mnist1L_sigm_Norm)[:,1], label=r'$L = 1$')
plt.plot(N_list,np.array(loss_mnist2L_sigm_Norm)[:,1], label=r'$L = 2$')
plt.plot(N_list,np.array(loss_mnist3L_sigm_Norm)[:,1], label=r'$L = 3$')
plt.xlabel(r'$N$')
#plt.yscale('log')


plt.legend()
#plt.tight_layout()


dataset="MNIST"
#plt.savefig('Error_{}_{}_NEW.pdf'.format(dataset,activ))
plt.show()

In [ ]:

from cycler import cycler

def alpha(alpha_relu,alpha_tanh,alpha_sigm):#,alpha_sigm):
    Layers=[1,2,3]
    c=1
    fig = plt.figure(figsize=(15,4)) 
    Ntrain=5000
    N_list=[np.int(rate*Ntrain) for rate in np.arange(0.1,1.1,0.1)]
    #N_list = np.array([50,100,200,400,800,1600])
    
    plt.rc('axes', prop_cycle=(cycler('color', ['tab:orange', 'tab:green', 'tab:red', 'tab:purple'])))
    plt.subplot(1, 3, c)
    #plt.title('ReLU')
    for alpha1min, L in zip(alpha_relu,Layers):
        alpha1norm=L*np.sum(np.sum(np.array(alpha1min)**2,axis=(1,2)),axis=1) # (10,) shape
        
        plt.plot(N_list,alpha1norm,label=r'$L = {}$'.format(L))
    plt.xlabel(r'$N$')
    #plt.yscale('log')
    c+=1
    
    plt.rc('axes', prop_cycle=(cycler('color', ['tab:orange', 'tab:green', 'tab:red', 'tab:purple'])))
    plt.subplot(1, 3, c)
    for alpha1min, L in zip(alpha_tanh,Layers):
        alpha1norm=L*np.sum(np.sum(np.array(alpha1min)**2,axis=(1,2)),axis=1) # (10,) shape
        plt.plot(N_list,alpha1norm,label=r'$L = {}$'.format(L))
    plt.xlabel(r'$N$')
    #plt.yscale('log')
    c+=1
    
    plt.rc('axes', prop_cycle=(cycler('color', ['tab:orange', 'tab:green', 'tab:red', 'tab:purple'])))
    plt.subplot(1, 3, c)
    for alpha1min, L in zip(alpha_sigm,Layers):
        alpha1norm=L*np.sum(np.sum(np.array(alpha1min)**2,axis=(1,2)),axis=1) # (10,) shape
        plt.plot(N_list,alpha1norm,label=r'$L = {}$'.format(L))
    plt.xlabel(r'$N$')
    #plt.yscale('log')
    c+=1
    
    #plt.subplot(1, 3, c)
    #for alpha1min, L in zip(alpha_sigm,Layers):
    #    alpha1norm=np.mean(np.sum(np.array(alpha1min)**2,axis=(1,2)),axis=1) # (10,) shape
    #    plt.plot(N_list,alpha1norm,label=r'$L = {}$'.format(L))
    #plt.xlabel(r'$N_{train}$')
    #c+=1
    
    plt.legend()

    #dataset="MNIST_fashion"

    #plt.savefig(f'alpha1_NEW_{dataset}.pdf')
    plt.show()

    #plt.tight_layout()

In [ ]:
alpha(alpha1min_list_fash_relu,alpha1min_list_fash_tanh,
      alpha1min_list_fash_sigm)#,alpha1min_list_synt_sigm)

In [ ]:
from matplotlib.colors import LogNorm

alpha1_avg=np.mean(np.array(alpha1_mnist3L_sigm_Norm)**2,axis=3) 


fig, axes = plt.subplots(1, 3)
fig.set_size_inches(15, 4)
for i, ax in zip(np.arange(1,10,4), axes):
    ax.imshow(np.log(alpha1_avg[i,:,:]), cmap='hot',interpolation='nearest')
    #,norm=LogNorm(vmin=1e-5, vmax=1))#interpolation='nearest')
    ax.set_title(f'$N={(i+1)*500}$')
    #ax.set_xticks([])
    #ax.set_yticks([])

#plt.tight_layout()
#plt.colorbar(im, cax=axes[4])
activ="sigmoid"
dataset="MNIST_fash"
#plt.savefig('heatplot3L_notLog{}_{}.pdf'.format(dataset,activ))
plt.show()

#c=1
#fig=plt.figure(figsize=(20,4)) 

#for i in np.arange(1,2,10):
#    plt.subplot(1,5,c)
#    plt.imshow(
#    c+=1
#    plt.colorbar()
#    plt.show()

In [ ]:
#%store -r alpha1min_list_fash_sigm

## 2) Regularization with CIFAR-10  (to complete)


In [ ]:
def LoadCIFAR():
    Ntrain=5000
    Ntest=10000
    cifar10 = tf.keras.datasets.cifar10
    class_names = ['plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train=x_train.astype('float64')
    x_test=x_test.astype('float64')
    x_train=x_train[0:Ntrain,:,:]
    y_train=y_train[0:Ntrain]
    x_test=x_test[0:Ntest,:,:]
    y_test=y_test[0:Ntest]
    Y_train=np.zeros((Ntrain,10))
    Y_test=np.zeros((Ntest,10))
    for i in range(10):
        Y_train[y_train[0:Ntrain,0]==i,i]=1
        Y_test[y_test[0:Ntest,0]==i,i]=1
    x_train, x_test = x_train / 255.0, x_test / 255.0
    normalizedata=False
    if normalizedata:
        a=np.sqrt(np.sum(x_train*x_train,(1,2)))
        a=a[:,None,None]+np.zeros((Ntrain,32,32))
        x_train = np.divide(x_train , a)
        a=np.sqrt(np.sum(x_test*x_test,(1,2)))
        a=a[:,None,None]+np.zeros((Ntest,32,32))
        x_test = np.divide(x_test , a)
        a=0 
    show=True
    if show == True:
        def showimages(x_train,y_train):
            plt.figure(figsize=(10,10))
            for i in range(25):
                plt.subplot(5,5,i+1)
                plt.xticks([])
                plt.yticks([])
                plt.grid(False)
                plt.imshow(x_train[i], cmap=plt.cm.binary)
                plt.xlabel(class_names[y_train[i,0]])
            plt.show()
        showimages(x_train,y_train)
    d2=32*32*3
    vx_train=np.reshape(x_train,(Ntrain,d2))
    vx_test=np.reshape(x_test,(Ntest,d2))
    return (vx_train,Y_train,vx_test,Y_test)

(X_train,Y_train,X_test,Y_test) = LoadCIFAR()

In [ ]:
Ntrain=5000
def get_sets_CIFAR(rate):
    vx_train=X_train[0:np.int(rate*Ntrain),:]
    y_train=Y_train[0:np.int(rate*Ntrain),:]

    vx_train=torch.tensor(vx_train.astype('float64'))
    vx_test=torch.tensor(X_test.astype('float64'))
    y_train=torch.tensor(y_train.astype('float64'))
    y_test=torch.tensor(Y_test.astype('float64'))

    return (vx_train, y_train), (vx_test, y_test)


rate=0.5
(vx_train, y_train), (vx_test, y_test) = get_sets_CIFAR(rate)

In [ ]:
def phi_reg(x,alpha1,z): #added slack variable z
    for i in range(L):
        xtopsi1_transp=torch.transpose(xtopsi1_torch(x),0,1)
        x=x+torch.matmul(xtopsi1_transp,alpha1[:,:,i])+z[:,:,i]
    return x

In [ ]:
def loss_reg(alpha1,z):
    loss1=torch.sum(alpha1*alpha1) + (1/r)*torch.sum(z*z) #update the loss 
    loss1=0.5*L*loss1
    phivx_train=phi_reg(vx_train,alpha1,z)
#    phivx_train=vx_train
    (loss2,alpha2)=getalpha2_torch(phivx_train,y_train)
    return nu*loss1+loss2

In [ ]:
def optim_alpha1_reg(rate,nsteps=1000,lr=0.1):
    alpha1_init=torch.zeros((D1,p,L),dtype=torch.float64)
    z_init=torch.zeros((np.int(rate*Ntrain),p,L),dtype=torch.float64)
    alpha1 = Variable(alpha1_init,requires_grad=True)
    z=Variable(z_init,requires_grad=True)
    opt = torch.optim.Adam([alpha1,z],lr=lr)

    loss_history = []

    for i in range(nsteps):
        opt.zero_grad()
        stdout.write("\r[%s]" % (i+1)) 
        loss = loss_reg(alpha1,z) # Calculate loss function
        loss_history.append(loss.detach().numpy()) # Do some recordings for plots
        
        loss.backward()
        opt.step()
        
       # if i % 50 == 0:
       #     print(f'iter {i+1}: loss = {loss}')
    
    #print(f'final loss: loss = {loss}')

    print(r"Min loss after training alpha and z: {}".format(np.min(loss_history)))

    plt.figure()    
    plt.plot(loss_history)                   #plot the loss as a function of iteration i
    plt.title('N= {}, lr= {}'.format(len(vx_train),lr))
    plt.show()
    
    return alpha1, z

        #stdout.write("\r[%s]" % loss) 




In [ ]:
gamma2=0.01
nu=0.01
r=0.01
L=1

p = vx_train.shape[1]
#D1 = p + 1
D1=200
#D2 = vx_train.shape[1]
D2=800


random1="true"
random2="true"
activ="relu"

Ntrain=5000


#alpha1_init=torch.zeros((D1,p,L),dtype=torch.float64)
#z_init=torch.zeros((np.int(rate*Ntrain),p,L),dtype=torch.float64)

rate=1
(vx_train, y_train), (vx_test, y_test) = get_sets_CIFAR(rate)

lr=0.0001
alpha1min, zmin = optim_alpha1_reg(rate=rate,nsteps=100,lr=lr)


In [ ]:
_,alpha2min=getalpha2_torch(phi_torch(vx_train,alpha1min),y_train)
_,alpha2_init=getalpha2_torch(vx_train,y_train)
        
inierror_train=get_error(vx_train,y_train,alpha2_init)#.numpy()
finerror_train=get_error(phi_torch(vx_train,alpha1min),y_train,alpha2min)#.detach().numpy()
        
inierror_test=get_error(vx_test,y_test,alpha2_init)#.numpy()
finerror_test=get_error(phi_torch(vx_test,alpha1min),y_test,alpha2min)#.detach().numpy()
        
print('initial {} error= {}'.format("train",np.round(inierror_train,3))) 
print('final {} error= {}'.format("train",np.round(finerror_train,3)))
    
print('initial {} error= {}'.format("test",np.round(inierror_test,3)))
print('final {} error= {}'.format("test",np.round(finerror_test,3)))




In [ ]:
x_train=np.reshape(phi_reg(vx_train[0:25,:],alpha1min,zmin[0:25,:,:]).detach().numpy(),(25,32,32,3))
x2=np.reshape(phi_torch(vx_train[0:25,:],torch.zeros((D1,p,L),dtype=torch.float64)).detach().numpy(),(25,32,32,3))
def showimagesCIFAR(x_train):
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(x_train[i], cmap=plt.cm.binary)
        #plt.title(gamma2)
    plt.show()
showimagesCIFAR(x2)
#showimagesCIFAR(x_train)